In [1]:
import sys
import random
import hashlib
import numpy as np
from os.path import abspath, join
sys.path.append(abspath(join('../..')))
sys.path.append(abspath(join('..')))

from math import pi

from LevelSetPy.Utilities import *
from LevelSetPy.BoundaryCondition import *
from LevelSetPy.Visualization import *
# from LevelSetPy.DynamicalSystems import *
from LevelSetPy.Grids import *
from LevelSetPy.DynamicalSystems import *
from LevelSetPy.InitialConditions import *
from LevelSetPy.SpatialDerivative import *
from LevelSetPy.ExplicitIntegration.Dissipation import *

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

In [2]:
# test a grid
gmin = np.array(([[-1, -1, -pi]]),dtype=np.float64).T
gmax = np.array(([[1, 1, pi]]),dtype=np.float64).T
grid = createGrid(gmin, gmax, 101, 2)

In [3]:

class Graph():
    def __init__(self, n, grids, vertex_set, edges):
        """A graph (an undirected graph that is) that models 
        the update equations of agents positions on a state space 
        (defined as a grid).

        The graph has a vertex set {1,2,...,n} so defined such that 
        (i,j) is one of the graph's edges in case i and j are neighbors.
        This graph changes over time since the relationship between neighbors
        can change.

        Paramters
        =========
            .grids
            n: number of initial birds (vertices) on this graph.
            .V: vertex_set, a set of vertices {1,2,...,n} that represent the labels
            of birds in a flock. Represent this as a list (see class vertex).
            .E: edges, a set of unordered pairs E = {(i,j): i,j \in V}.
                Edges have no self-loops i.e. i≠j or repeated edges (i.e. elements are distinct).
        """
        self.N = n
        if vertex_set is None:            
            self.vertex_set = {f"{i+1}":BirdSingle(grids[i], 1, 1,\
                    None, random.random(), label=f"{i}") for i in range(n)}
        else:
            self.vertex_set = vertex_set
        self.edges_set = edges 

        # obtain the graph params
        self.reset()

    def reset(self):
        # graph entities: this from Jadbabaie's paper
        self.Ap = np.zeros((self.N, self.N)) #adjacency matrix
        self.Dp = np.zeros((self.N, self.N)) #diagonal matrix of valencies
        self.θs = np.zeros((self.N, 1)).fill(self.w(1)) # agent headings
        self.I  = np.ones((self.num_birds, self.num_birds))
        self.Fp = np.zeros_like(self.Ap) # transition matrix for all the headings in this flock

    def insert_vertex(self, vertex):
        if isinstance(vertex, list):
            assert isinstance(vertex, Vertex), "vertex to be inserted must be instance of class Vertex."
            for vertex_single in vertex:
                self.vertex_set[vertex_single.label] = vertex_single.neighbors
        else:
            self.vertex_set[vertex.label] = vertex

    def insert_edge(self, from_vertex, to_vertex):
        assert isinstance(from_vertex, Vertex), "from_vertex to be inserted must be instance of class Vertex."
        assert isinstance(to_vertex, Vertex), "to_vertex to be inserted must be instance of class Vertex."
        from_vertex.update_neighbor(to_vertex)
        self.vertex_set[from_vertex.label] = from_vertex.neighbors
        self.vertex_set[to_vertex.label] = to_vertex.neighbors

    def insert_edges(self, from_vertices, to_vertices):
        for from_vertex, to_vertex in zip(from_vertices, to_vertices):
            self.insert_edge(from_vertex, to_vertex)

    def adjacency_matrix(self, t):
        # if len(self.vertex_set)>1:
        for i in range(self.Ap.shape[0]):
            for j in range(self.Ap.shape[1]):
                for verts in sorted(self.vertex_set.keys()):
                    if str(j) in self.vertex_set[verts].neighbors:
                        self.Ap[i,j] = 1 
        return self.Ap

    def diag_matrix(self):
        "build Dp matrix"
        i=0
        for vertex, egdes in self.vertex_set.items():
            self.Dp[i,i] = self.vertex_set[vertex].valence
        return self.Dp

    def update_headings(self, t):
        return self.adjacency_matrix(t)@self.θs



In [4]:
'we will first create the grids coverage for the respective birds'

num_agents=10

# for every agent, create the grid bounds
grid_mins = [[-1, -1, -np.pi]]
grid_maxs = [[1, 1, np.pi]]   
# grids = [createGrid(np.asarray(grid_mins), np.asarray(grid_maxs), N=100, pdDims=2)]

grids = flockGrid(grid_mins, grid_maxs, dx=.1, N=101)

# sanity check
grids[-1].N.T

array([[101, 101, 101]])

In [5]:
import time
import random
import hashlib
import cupy as cp
import numpy as np
from LevelSetPy.Utilities import eps


class BirdSingle():
    def __init__(self, grid, u_bound=+1, w_bound=+1, \
                 init_state=None, rw_cov=None, \
                 axis_align=2, center=None, 
                 neigh_rad=.3, init_random=False,
                 label="0", neighbors=[]):
        """
            Dubins Vehicle Dynamics in absolute coordinates.
            Please consult Merz, 1972 for a detailed reference.

            Dynamics:
            ==========
                \dot{x}_1 = v cos x_3
                \dot{x}_2 = v sin x_3
                \dot{x}_3 = w

            Parameters:
            ===========
                grid: an np.meshgrid state space on which we are
                resolving this vehicular dynamics. This grid does not have
                a value function (yet!) until it's part of a flock
                u_bound: absolute value of the linear speed of the vehicle.
                w_bound: absolute value of the angular speed of the vehicle.
                init_state: initial position and orientation of a bird on the grid
                rw_cov: random covariance scalar for initiating the stochasticity
                on the grid.
                center: location of this bird's value function on the grid
                axis_align: periodic dimension on the grid to be created
                neigh_rad: sets of neighbors of agent i
                label: label of this bird; must be a unique integer
                n: number of neighbors of this agent at time t
                init_random: randomly initialize this agent's position on the grid?

            BirdSingle Parameters
            =================
                .label (str): The label of this BirdSingle drawn from the set {1,2,...,n} 
                .valence(int): number of edges incident on a BirdSingle V_i
                .indicant_edge(tuple): an edge (i,j) such that i=v or j=v.
                .neighbors: neighbors of this BirdSingle (as labels).

            Test
            ====
                b0 = BirdSingle(.., label="0")
                b1 = BirdSingle(.., "1")
                b2 = BirdSingle(.., "2")
                b3 = BirdSingle(.., "3")
                b0.update_neighbor(b1)
                b0.update_neighbor(b2)
                b2.update_neighbor(b3)
                print(b0)
                print(b1)
                print(b2)
                print(b3)

                Prints: BirdSingle: 0 | Neighbors: ['1', '2']
                        BirdSingle: 1 | Neighbors: ['0']
                        BirdSingle: 2 | Neighbors: ['0', '3']
                        BirdSingle: 3 | Neighbors: ['2']

                Multiple neighbors test
                -----------------------
                neighs_2 = [BirdSingle(f"{i}") for i in range(3, 9)]
                b2.update_neighbors(neighs_2)
                print(b2)

                Prints
        """

        assert label is not None, "label of an agent cannot be empty"
        # BirdSingle Params
        self.label = label
        # set of labels of those agents whicvh are neighbors of this agent
        self.neighbors   = neighbors
        self.inidicant_edge = []       
        # minimum L2 distance that defines a neighbor 
        self.neigh_rad = neigh_rad
        self.init_random = init_random
        self.n  = 0

        # grid params
        self.grid        = grid
        self.center      = center
        self.axis_align  = axis_align

        # linear and angular velocities of the bird
        self.v = u_bound
        self.w = w_bound

        # this is a vector defined in the direction of its nearest neighbor
        self.u = None
        self.deltaT = eps # use system eps for a rough small start due to in deltaT
        self.rand_walk_cov = random.random if rw_cov is None else rw_cov

        # position this bird at the center of the state space
        self.position(init_state)
        
    def position(self, cur_state=None, t_span=None):
        """
            Birds use an optimization scheme to keep
            separated distances from one another.

            'even though vision is the main mechanism of interaction,
            optimization determines the anisotropy of neighbors, and
            not the eye's structure. There is also the possibility that
            each individual keeps the front neighbor at larger distances
            to avoid collisions. This collision avoidance mechanism is
            vision-based but not related to the eye's structure.'

            Parameters
            ==========
            cur_state: position and orientation.
                i.e. [x1, x2, θ] at this current position
            t_span: time_span as a list [t0, tf] where
                .t0: initial integration time
                .tf: final integration time
            
            Parameters
            ==========
            .init_state: current state of a bird in the state space
                (does not have to be an initial state/could be a current
                state during simulation). If it is None, it is initialized
                on the center of the state space.
        """
        if not np.any(cur_state):
            #put cur_state at origin if not specified
            cur_state = [np.mean(self.grid.vs[0]),
                         np.mean(self.grid.vs[1]),
                         np.mean(self.grid.vs[2])
                         ]
        
        X = self.do_runge_kutta4(cur_state, t_span)

        if self.init_random:
            # Simulate each agent's position in a flock as a random walk
            W = np.asarray(([self.deltaT**2/2])).T
            WW = W@W.T
            rand_walker = np.ones((len(cur_state))).astype(float)*WW*self.rand_walk_cov**2

            X += rand_walker

        return X

    def dynamics(self, cur_state):
        """
            Computes the Dubins vehicular dynamics in relative
            coordinates (deterministic dynamics).

            \dot{x}_1 = -v_e + v_p cos x_3 + w_e x_2
            \dot{x}_2 = -v_p sin x_3 - w_e x_1
            \dot{x}_3 = -w_p - w_e
        """
        xdot = [
                self.v * np.cos(cur_state[2]),
                self.v * np.sin(cur_state[2]),
                self.w
        ]

        return np.asarray(xdot)

    def update_neighbor(self, neigh):
        if isinstance(neigh, list):
            for neigh_single in neigh:
                self.update_neighbor(neigh_single)
            return
        assert isinstance(neigh, BirdSingle), "Neighbor must be a BirdSingle member function."
        # assert neigh not in self.neighbors, "No repeated neighbors allowed."
        # assert neigh!=self, "Cannot assign a BirdSingle as its own neighbor"

        if neigh in self.neighbors or neigh==self:
            return self.neighbors 

        self.neighbors.append(neigh)

        # this neighbor must be a neighbor of this parent
        neigh.neighbors.append(self) 

    @property
    def valence(self):
        """
            By how much has the number of edges incident
            on v changed?

            Parameter
            =========
            delta: integer (could be positive or negative).

            It is positive if the number of egdes increases at a time t.
            It is negative if the number of egdes decreases at a time t.
        """
        return len(self.neighbors)
    
    def update_inidicant_edges(self, edges):
        """
            Update the number of edges (i,j) of the graph for which either
            i=j or j=v.
        """   
        pass     

    def do_runge_kutta4(self, cur_state, t_span, M=4, h=2):
        """
            .cur_state: state at time space 
            .t_span
            .M: RK steps per interval 
            .h: time step        
        """
        # integrate the dynamics with 4th order RK scheme
        X = np.array(cur_state) if isinstance(cur_state, list) else cur_state

        for j in range(M):
            if np.any(t_span): # integrate for this much time steps
                hh = (t_span[1]-t_span[0])/10/M
                for h in np.arange(t_span[0], t_span[1], hh):
                    k1 = self.dynamics(X)
                    k2 = self.dynamics(X + h/2 * k1)
                    k3 = self.dynamics(X + h/2 * k2)
                    k4 = self.dynamics(X + h * k3)

                    X  = X+(h/6)*(k1 + 2*k2 + 2*k3 + k4)
            else:
                k1 = self.dynamics(X)
                k2 = self.dynamics(X + h/2 * k1)
                k3 = self.dynamics(X + h/2 * k2)
                k4 = self.dynamics(X + h * k3)

                X  = X+(h/6)*(k1 +2*k2 +2*k3 +k4)

        return X

    def __hash__(self):
        # hash method to distinguish agents from one another    
        return int(hashlib.md5(str(self.label).encode('utf-8')).hexdigest(),16)

    def __eq__(self,other):
        if hash(self)==hash(other):
            return True
        return False

    def __repr__(self):
        s="Bird {}."\
        .format(self.label)
        return s  
        parent=f"BirdSingle: {self.label} | "
        children="Neighbors: 0" if not self.neighbors \
                else f"Neighbors: {sorted([x.label for x in self.neighbors])}"
        return parent + children  

In [6]:
init_state = np.zeros(grid.shape)
ref_bird = BirdSingle(grid, 1, 1, None, \
                    random.random(), label=0) 

In [7]:
ref_bird.position()

array([0.99558554, 1.15271013, 8.        ])

In [8]:
__all__ = ["BirdFlock"]

__author__ = "Lekan Molux"
__date__ = "Dec. 21, 2021"
__comment__ = "Two Dubins Vehicle in Relative Coordinates"

import hashlib
import cupy as cp
import numpy as np
import random
from .bird_single_leaderless import BirdSingle
from LevelSetPy.Utilities.matlab_utils import *


class Graph():
    def __init__(self, n, grids, vertex_set, edges=None):
        """A graph (an undirected graph that is) that models 
        the update equations of agents positions on a state space 
        (defined as a grid).

        The graph has a vertex set {1,2,...,n} so defined such that 
        (i,j) is one of the graph's edges in case i and j are neighbors.
        This graph changes over time since the relationship between neighbors
        can change.

        Paramters
        =========
            .grids
            n: number of initial birds (vertices) on this graph.
            .V: vertex_set, a set of vertices {1,2,...,n} that represent the labels
            of birds in a flock. Represent this as a list (see class vertex).
            .E: edges, a set of unordered pairs E = {(i,j): i,j \in V}.
                Edges have no self-loops i.e. i≠j or repeated edges (i.e. elements are distinct).
        """
        self.N = n
        if vertex_set is None:            
            self.vertex_set = {f"{i+1}":BirdSingle(grids[i], 1, 1,\
                    None, random.random(), label=f"{i}") for i in range(n)}
        else:
            self.vertex_set = {f"{i+1}":vertex_set[i] for i in range(n)}
        
        # edges are updated dynamically during game
        self.edges_set = edges 

        # obtain the graph params
        self.reset()

    def reset(self):
        # graph entities: this from Jadbabaie's paper
        self.Ap = np.zeros((self.N, self.N)) #adjacency matrix
        self.Dp = np.zeros((self.N, self.N)) #diagonal matrix of valencies
        self.θs = np.zeros((self.N, 1)).fill(self.w(1)) # agent headings
        self.I  = np.ones((self.num_birds, self.num_birds))
        self.Fp = np.zeros_like(self.Ap) # transition matrix for all the headings in this flock

    def insert_vertex(self, vertex):
        if isinstance(vertex, list):
            assert isinstance(vertex, BirdSingle), "vertex to be inserted must be instance of class Vertex."
            for vertex_single in vertex:
                self.vertex_set[vertex_single.label] = vertex_single.neighbors
        else:
            self.vertex_set[vertex.label] = vertex

    def insert_edge(self, from_vertices, to_vertices):
        if isinstance(from_vertices, list) and isinstance(to_vertices, list):
            for from_vertex, to_vertex in zip(from_vertices, to_vertices):
                self.insert_edge(from_vertex, to_vertex)
            return
        else:
            assert isinstance(from_vertices, BirdSingle), "from_vertex to be inserted must be instance of class Vertex."
            assert isinstance(to_vertices, BirdSingle), "to_vertex to be inserted must be instance of class Vertex."
            from_vertices.update_neighbor(to_vertices)
            self.vertex_set[from_vertices.label] = from_vertices.neighbors
            self.vertex_set[to_vertices.label] = to_vertices.neighbors

    def adjacency_matrix(self, t):
        for i in range(self.Ap.shape[0]):
            for j in range(self.Ap.shape[1]):
                for verts in sorted(self.vertex_set.keys()):
                    if str(j) in self.vertex_set[verts].neighbors:
                        self.Ap[i,j] = 1 
        return self.Ap

    def diag_matrix(self):
        "build Dp matrix"
        i=0
        for vertex, egdes in self.vertex_set.items():
            self.Dp[i,i] = self.vertex_set[vertex].valence
        return self.Dp

    def update_headings(self, t):
        return self.adjacency_matrix(t)@self.θs



class BirdFlock(BirdSingle):
    def __init__(self, grids, u_bound=1, w_bound=1, num_agents=7, 
                reach_rad=1.0, avoid_rad=1.0):
        """
            A flock of Dubins Vehicles. These are patterned after the 
            behavior of starlings which self-organize into local flocking patterns.

            Note that here, we must work in absolute coordinates.

            The inspiration for this is the following paper:

                "Interaction ruling animal collective behavior depends on topological 
                rather than metric distance: Evidence from a field study." 
                ~ Ballerini, Michele, Nicola Cabibbo, Raphael Candelier, 
                Andrea Cavagna, Evaristo Cisbani, Irene Giardina, Vivien Lecomte et al. 
                Proceedings of the national academy of sciences 105, no. 4 
                (2008): 1232-1237. 

            Dynamics:
                \dot{x}_1 = -v_e + v_p cos x_3 + w_e x_2
                \dot{x}_2 = -v_p sin x_3 - w_e x_1
                \dot{x}_3 = -w_p - w_e

            Parameters
            ==========
                .grids: 2 possible types of grids exist for resolving vehicular dynamics:
                    .single_grid: an np.meshgrid that homes all these birds
                    .multiple grids: a collection of possibly intersecting grids 
                    where agents interact.                
                .u_bound: absolute value of the linear speed of the vehicle.
                .w_bound: absolute value of the angular speed of the vehicle.
                .num_agents: number of agents in this flock of vehicles.
                .center: center of the target set.
                .radius of the target set's grom primitive.
        """

        # for the nearest neighors in this flock, they should have an anisotropic policy
        # set linear speeds
        if not np.isscalar(u_bound) and len(u_bound) > 1:
            self.v_e = self.v(1)
            self.v_p = self.v(-1)
        else:
            self.v_e = self.v(1)
            self.v_p = self.v(1)

        # set angular speeds
        if not np.isscalar(w_bound) and len(w_bound) > 1:
            self.w_e = self.w(1)
            self.w_p = self.w(-1)
        else:
            self.w_e = self.w(1)
            self.w_p = self.w(1)
            
        # Number of vehicles in this flock
        self.N           = num_agents

        # birds could be on different subspaces of an overall grid
        if isinstance(grids, list):
            self.vehicles = []
            #reference bird must be at origin of the grid
            bird_pos = [ 
                         np.mean(grids[0].vs[0]),
                         np.mean(grids[0].vs[1]),
                         np.mean(grids[0].vs[2])
                         ]
            lab = 0
            for each_grid in grids:
                self.vehicles.append(BirdSingle(each_grid, self.v_e, self.w_e, \
                                        bird_pos, label=lab, neighbors=[]))
                # randomly initialize position of other birds
                bird_pos = [np.random.sample(each_grid.vs[0], 1), \
                            np.random.sample(each_grid.vs[1], 1), \
                            np.random.sample(each_grid.vs[2], 1)]
                lab += 1
        else: # all birds are on the same grid
            self.vehicles = [BirdSingle(grids, self.v_e, self.w, \
                                [np.random.sample(grids.vs[0], 1), \
                                 np.random.sample(grids.vs[1], 1), \
                                 np.random.sample(grids.vs[2], 1)], \
                                random.random(), label=_+1) for _ in range(num_agents)]

        self.grid = grids
        """
             Define the anisotropic parameter for this flock.
             This gamma parameter controls the degree of interaction among 
             the agents in this flock. Interaction decays with the distance, and 
             we can use the anisotropy to get information about the interaction.
             Note that if nc=1 below, then the agents 
             exhibit isotropic behavior and the aggregation is non-interacting by and large.
        """
        self.gamma = lambda nc: (1/3)*nc

        """create the target set for this local flock"""
        self.flock_payoff = self.get_target(reach_rad, avoid_rad)

        # recursively update each agent's position and neighbors in the state
        for agent in self.vehicles:
            self.update_agent_single(agent)      

        # write the graph lookup of these flock's positions
        self.graph = Graph(num_agents, self.grid, self.vehicles, None)  

    def update_agent_single(self, agent, t=None):
        """
            Compute the # of neighbors of this `agent`, labels of the
            neighbors of this agent as a list, and update the average
            heading of this agent -- all at time t

            Update the number of nearest neighbors of this agent
            and then the labels of its neighbors for bookkeeping.

            Parameters:
            ==========
            agent: This agent as a BirdsSingle object.
            t: Time at which we are updating this agent's dynamics.
        """
        # neighbors are those agents within a normed distance to this agent's position
        n    = agent.n
        label = agent.label # we do not expect this to change

        # update headings and neighbors (see eqs 1 and 2 in Jadbabaie)
        for other_agent in self.vehicles:
            if other_agent == agent:
                # we only compare with other agents
                continue 

            # TODO: how to better find a vehicle's position on the state space at t? 
            dist = np.norm(other_agent.position()[:2], 2) - np.norm(agent.position()[:2], 2)
            if dist <= agent.neigh_rad:
                n += 1 # increase neighbor count if we are within the prespecified radius
                agent.neighbors.append(other_agent.label) # label will be integers
        
        # update heading for this agent
        if np.any(agent.neighbors):
            neighbor_headings = [self.vehicles[agent.neighbors[i]].w \
                for i in range(len(agent.neighbors)) \
                    if agent!=self.vehicles[agent.neighbors[i]]]
        else:
            neighbor_headings = 0
        # this maps headings w/values in [0, 2\pi) to [0, \pi)
        θr = (1/(1+n))*(agent.w + np.sum(neighbor_headings))   

    def get_target(self, reach_rad=1.0, avoid_rad=1.0):
        """
            Make reference bird the evader and every other bird the pursuer
            owing to the lateral visual anisotropic characteric of starlings.
        """
        # first bird is the evader, so collect its position info
        evader = self.vehicles[0]
        target_set = np.zeros((self.N-1,)+(evader.grid.shape), dtype=np.float64)
        payoff_capture = np.zeros((evader.grid.shape), dtype=np.float64)
        # first compute the any pursuer captures an evader
        for pursuer in self.vehicles[1:]:
            if not np.any(pursuer.center):
                pursuer.center = np.zeros((pursuer.grid.dim, 1))
            elif(numel(pursuer.center) == 1):
                pursuer.center = pursuer.center * np.ones((pursuer.grid.dim, 1), dtype=np.float64)

            #---------------------------------------------------------------------------
            #axis_align must be same for all agents in a flock
            # any pursuer can capture the reference bird
            for i in range(pursuer.grid.dim):
                if(i != pursuer.axis_align):
                    target_set[cur_agent] += (pursuer.grid.xs[i] - evader.grid.xs[i])**2
            target_set[cur_agent] = np.sqrt(target_set[cur_agent])

            # take an element wise min of all corresponding targets now
            if cur_agent >= 1:
                payoff_capture = np.minimum(target_set[cur_agent], target_set[cur_agent-1], dtype=np.float64)
            cur_agent += 1
        payoff_capture -= reach_rad

        # compute the anisotropic value function: this maintains the gap between the pursuers
        target_set = np.zeros((self.N-1,)+(evader.grid.shape), dtype=np.float64)
        payoff_avoid = np.zeros((evader.grid.shape), dtype=np.float64)
        cur_agent = 0
        for vehicle_idx in range(1, len(self.vehicles)-1):
            this_vehicle = self.vehicles[vehicle_idx]
            next_vehicle = self.vehicles[vehicle_idx+1]
            for i in range(this_vehicle.grid.dim):
                if(i != this_vehicle.axis_align):
                    target_set[cur_agent] += (this_vehicle.grid.xs[i] + next_vehicle.grid.xs[i])**2
            target_set[cur_agent] = np.sqrt(target_set)

            # take an element wise min of all corresponding targets now
            if cur_agent >= 1:
                payoff_avoid = np.minimum(target_set[cur_agent], target_set[cur_agent-1], dtype=np.float64)
            cur_agent += 1
        
        payoff_avoid -= avoid_rad

        # now do a union of both the avoid and capture sets
        combo_payoff = np.minimum(payoff_avoid, payoff_capture)

        return combo_payoff



    def hamiltonian(self, t, data, value_derivs, finite_diff_bundle):
        """
            H = p_1 [v_e - v_p cos(x_3)] - p_2 [v_p sin x_3] \
                   - w | p_1 x_2 - p_2 x_1 - p_3| + w |p_3|

            Parameters
            ==========
            value: Value function at this time step, t
            value_derivs: Spatial derivatives (finite difference) of
                        value function's grid points computed with
                        upwinding.
            finite_diff_bundle: Bundle for finite difference function
                .innerData: Bundle with the following fields:
                    .partialFunc: RHS of the o.d.e of the system under consideration
                        (see function dynamics below for its impl).
                    .hamFunc: Hamiltonian (this function).
                    .dissFunc: artificial dissipation function.
                    .derivFunc: Upwinding scheme (upwindFirstENO2).
                    .innerFunc: terminal Lax Friedrichs integration scheme.
        """
        p1, p2, p3 = value_derivs[0], value_derivs[1], value_derivs[2]
        p1_coeff = self.v_e - self.v_p * cp.cos(self.grid.xs[2])
        p2_coeff = self.v_p * cp.sin(self.grid.xs[2])

        Hxp = p1 * p1_coeff - p2 * p2_coeff - self.w(1)*cp.abs(p1*self.grid.xs[1] - \
                p2*self.grid.xs[0] - p3) + self.w(1) * cp.abs(p3)

        return Hxp

    def dissipation(self, t, data, derivMin, derivMax, \
                      schemeData, dim):
        """
            Parameters
            ==========
                dim: The dissipation of the Hamiltonian on
                the grid (see 5.11-5.12 of O&F).

                t, data, derivMin, derivMax, schemeData: other parameters
                here are merely decorators to  conform to the boilerplate
                we use in the levelsetpy toolbox.
        """
        assert dim>=0 and dim <3, "Dubins vehicle dimension has to between 0 and 2 inclusive."

        if dim==0:
            return cp.abs(self.v_e - self.v_p * cp.cos(self.grid.xs[2])) + cp.abs(self.w(1) * self.grid.xs[1])
        elif dim==1:
            return cp.abs(self.v_p * cp.sin(self.grid.xs[2])) + cp.abs(self.w(1) * self.grid.xs[0])
        elif dim==2:
            return self.w_e + self.w_p

    def dynamics(self):
        """
            Computes the Dubins vehicular dynamics in relative
            coordinates (deterministic dynamics).

            \dot{x}_1 = -v_e + v_p cos x_3 + w_e x_2
            \dot{x}_2 = -v_p sin x_3 - w_e x_1
            \dot{x}_3 = -w_p - w_e
        """
        x1 = self.grid.xs[0]
        x2 = self.grid.xs[1]
        x3 = self.grid.xs[2]

        xdot = [
                -self.ve + self.vp * np.cos(x3) + self.we * x2,
                -self.vp * np.sin(x3) - self.we * x1,
                -self.wp - self.we # pursuer minimizes
        ]

        return xdot

ImportError: attempted relative import with no known parent package

In [ ]:
'we will first create the grids coverage for the respective birds'

num_agents=10

# for every agent, create the grid bounds
grid_mins = [[-1, -1, -np.pi]]
grid_maxs = [[1, 1, np.pi]]   
grids = flockGrid(grid_mins, grid_maxs, dx=.1, N=101)

# sanity check
grids[-1].N.T

array([[101, 101, 101]])

In [ ]:
print(ref_bird)  

neighs = [BirdSingle(grids[i], 1, 1, None, \
                    random.random(), label=i) for i in range(num_agents)]
ref_bird.update_neighbor(neighs)
print(ref_bird, ' || valence: ', ref_bird.valence)

In [30]:

class BirdFlock(BirdSingle):
    def __init__(self, grids, num_agents=7, grid_nodes=101,
                reach_rad=1.0, avoid_rad=1.0):
        """
            A flock of Dubins Vehicles. These are patterned after the 
            behavior of starlings which self-organize into local flocking patterns.

            Note that here, we must work in absolute coordinates.

            The inspiration for this is the following paper:

                "Interaction ruling animal collective behavior depends on topological 
                rather than metric distance: Evidence from a field study." 
                ~ Ballerini, Michele, Nicola Cabibbo, Raphael Candelier, 
                Andrea Cavagna, Evaristo Cisbani, Irene Giardina, Vivien Lecomte et al. 
                Proceedings of the national academy of sciences 105, no. 4 
                (2008): 1232-1237. 

            Dynamics:
                \dot{x}_1 = -v_e + v_p cos x_3 + w_e x_2
                \dot{x}_2 = -v_p sin x_3 - w_e x_1
                \dot{x}_3 = -w_p - w_e

            Parameters
            ==========
                .grids: 2 possible types of grids exist for resolving vehicular dynamics:
                    .single_grid: an np.meshgrid that homes all these birds
                    .multiple grids: a collection of possibly intersecting grids 
                    where agents interact.
                
                .num_agents: number of agents in this flock of vehicles.

                .grid_nodes: number of nodes in grid.
        """
        # Number of vehicles in this flock
        self.N  = num_agents
        self.grid_nodes = grid_nodes

        if grids is None and num_agents==1:
            # for every agent, create the grid bounds
            grid_mins = [[-1, -1, -np.pi]]
            grid_maxs = [[1, 1, np.pi]]   
            grids = flockGrid(grid_mins, grid_maxs, dx=.1, num_agents=num_agents, N=grid_nodes)
        elif grids is None and num_agents>1:
            gmin = np.array(([[-1, -1, -np.pi]]),dtype=np.float64).T
            gmax = np.array(([[1, 1, np.pi]]),dtype=np.float64).T
            grid = createGrid(gmin, gmax, grid_nodes, 2)

        # birds could be on different subspaces of an overall grid
        if isinstance(grids, list):
            self.vehicles = []
            #reference bird must be at origin of the grid
            # bird_pos = [ 
            #              np.mean(grids[0].vs[0]),
            #              np.mean(grids[0].vs[1]),
            #              np.mean(grids[0].vs[2])
            #              ]
            lab = 0
            for each_grid in grids:
                self.vehicles.append(BirdSingle(each_grid,1,1,None, \
                                         random.random(), label=lab))
                # # randomly initialize position of other birds
                # bird_pos = [np.random.sample(each_grid.vs[0], 1), \
                #             np.random.sample(each_grid.vs[1], 1), \
                #             np.random.sample(each_grid.vs[2], 1)]
                lab += 1
        else: # all birds are on the same grid
            ref_bird = BirdSingle(grids[0], 1, 1, None, \
                                random.random(), label=0) 
            self.vehicles = [BirdSingle(grids[i], 1, 1, None, \
                    random.random(), label=i) for i in range(1,num_agents)]

        self.grid = grids
        """
             Define the anisotropic parameter for this flock.
             This gamma parameter controls the degree of interaction among 
             the agents in this flock. Interaction decays with the distance, and 
             we can use the anisotropy to get information about the interaction.
             Note that if nc=1 below, then the agents 
             exhibit isotropic behavior and the aggregation is non-interacting by and large.
        """
        self.gamma = lambda nc: (1/3)*nc
        """create the target set for this local flock"""
        self.flock_payoff = self.get_target(reach_rad=1.0, avoid_rad=1.0)
            
        self.graph = Graph(num_agents, self.grid, self.vehicles, None)

    def update_dynamics(self):
        """
            Update the dynamics of the agents on this grid whose positionings are
            determined by a graph self.graph.
        """
        
        # recursively update each agent's position and neighbors in the state
        for idx in range(len(self.vehicles)):
            self.graph.θs[idx,:] = self.update_agent_single(self.vehicles[idx])

    def update_agent_single(self, agent, t=None):
        """
            Compute the # of neighbors of this `agent` at time t.
            In addition, update the number of neighbors of this agent
             as labels in a list, and compute the average heading of
             this robot as well. 

            Update the number of nearest neighbors of this agent
            and then the labels of its neighbors for bookkeeping.

            Parameters:
            ==========
            agent: This agent as a BirdsSingle object.
            t: Time at which we are updating this agent's dynamics.
        """
        # neighbors are those agents within a normed distance to this agent's position
        n    = agent.n
        label = agent.label # we do not expect this to change

        # update headings and neighbors (see eqs 1 and 2 in Jadbabaie)
        for other_agent in self.vehicles:
            if other_agent == agent:
                # we only compare with other agents
                continue 

            # TODO: how to better find a vehicle's position on the state space at t? 
            dist = np.linalg.norm(other_agent.position()[:2], 2) - np.linalg.norm(agent.position()[:2], 2)
            if dist <= agent.neigh_rad:
                n += 1 # increase neighbor count if we are within the prespecified radius
                agent.neighbors.append(other_agent.label) # label will be integers
        
        # update heading for this agent
        if np.any(agent.neighbors):
            neighbor_headings = [self.vehicles[agent.neighbors[i]].w \
                for i in range(len(agent.neighbors)) \
                    if agent!=self.vehicles[agent.neighbors[i]]]
        else:
            neighbor_headings = 0
        # this maps headings w/values in [0, 2\pi) to [0, \pi)
        θr = (1/(1+n))*(agent.w + np.sum(neighbor_headings))        
        #agent.update_agent_params(t, n, label, θr)
        return θr
           
    def get_target(self, reach_rad=1.0, avoid_rad=1.0):
        """
            Make reference bird the evader and every other bird the pursuer
            owing to the lateral visual anisotropic characteric of starlings.
        """
        # first bird is the evader, so collect its position info
        cur_agent = 0
        evader = self.vehicles[cur_agent]
        target_set = np.zeros((self.N-1,)+(evader.grid.shape), dtype=np.float64)
        payoff_capture = np.zeros((evader.grid.shape), dtype=np.float64)
        # first compute the any pursuer captures an evader
        for pursuer in self.vehicles[1:]:
            if not np.any(pursuer.center):
                pursuer.center = np.zeros((pursuer.grid.dim, 1))
            elif(numel(pursuer.center) == 1):
                pursuer.center = pursuer.center * np.ones((pursuer.grid.dim, 1), dtype=np.float64)

            #---------------------------------------------------------------------------
            #axis_align must be same for all agents in a flock
            # any pursuer can capture the reference bird
            for i in range(pursuer.grid.dim):
                if(i != pursuer.axis_align):
                    target_set[cur_agent] += (pursuer.grid.xs[i] - evader.grid.xs[i])**2
            target_set[cur_agent] = np.sqrt(target_set[cur_agent])

            # take an element wise min of all corresponding targets now
            if cur_agent >= 1:
                payoff_capture = np.minimum(target_set[cur_agent], target_set[cur_agent-1], dtype=np.float64)
            cur_agent += 1
        payoff_capture -= reach_rad

        # compute the anisotropic value function: this maintains the gap between the pursuers
        # note this is also the avoid set
        target_set = np.zeros((self.N-1,)+(evader.grid.shape), dtype=np.float64)
        payoff_avoid = np.zeros((evader.grid.shape), dtype=np.float64)
        cur_agent = 0
        for vehicle_idx in range(1, len(self.vehicles)-1):
            this_vehicle = self.vehicles[vehicle_idx]
            next_vehicle = self.vehicles[vehicle_idx+1]
            for i in range(this_vehicle.grid.dim):
                if(i != this_vehicle.axis_align):
                    target_set[cur_agent] += (this_vehicle.grid.xs[i] + next_vehicle.grid.xs[i])**2
            target_set[cur_agent] = np.sqrt(target_set[cur_agent])

            # take an element wise min of all corresponding targets now
            if cur_agent >= 1:
                payoff_avoid = np.minimum(target_set[cur_agent], target_set[cur_agent-1], dtype=np.float64)
            cur_agent += 1
        
        payoff_avoid -= avoid_rad

        # now do a union of both the avoid and capture sets
        combo_payoff = np.minimum(payoff_avoid, payoff_capture)

        return combo_payoff

    def hamiltonian(self, t, data, value_derivs, finite_diff_bundle):
        """
            H = p_1 [v_e - v_p cos(x_3)] - p_2 [v_p sin x_3] \
                   - w | p_1 x_2 - p_2 x_1 - p_3| + w |p_3|

            Parameters
            ==========
            value: Value function at this time step, t
            value_derivs: Spatial derivatives (finite difference) of
                        value function's grid points computed with
                        upwinding.
            finite_diff_bundle: Bundle for finite difference function
                .innerData: Bundle with the following fields:
                    .partialFunc: RHS of the o.d.e of the system under consideration
                        (see function dynamics below for its impl).
                    .hamFunc: Hamiltonian (this function).
                    .dissFunc: artificial dissipation function.
                    .derivFunc: Upwinding scheme (upwindFirstENO2).
                    .innerFunc: terminal Lax Friedrichs integration scheme.
        """
        p1, p2, p3 = value_derivs[0], value_derivs[1], value_derivs[2]
        p1_coeff = self.v_e - self.v_p * cp.cos(self.grid.xs[2])
        p2_coeff = self.v_p * cp.sin(self.grid.xs[2])

        Hxp = p1 * p1_coeff - p2 * p2_coeff - self.w(1)*cp.abs(p1*self.grid.xs[1] - \
                p2*self.grid.xs[0] - p3) + self.w(1) * cp.abs(p3)

        return Hxp

    def dissipation(self, t, data, derivMin, derivMax, \
                      schemeData, dim):
        """
            Parameters
            ==========
                dim: The dissipation of the Hamiltonian on
                the grid (see 5.11-5.12 of O&F).

                t, data, derivMin, derivMax, schemeData: other parameters
                here are merely decorators to  conform to the boilerplate
                we use in the levelsetpy toolbox.
        """
        assert dim>=0 and dim <3, "Dubins vehicle dimension has to between 0 and 2 inclusive."

        if dim==0:
            return cp.abs(self.v_e - self.v_p * cp.cos(self.grid.xs[2])) + cp.abs(self.w(1) * self.grid.xs[1])
        elif dim==1:
            return cp.abs(self.v_p * cp.sin(self.grid.xs[2])) + cp.abs(self.w(1) * self.grid.xs[0])
        elif dim==2:
            return self.w_e + self.w_p


In [31]:

class Graph():
    def __init__(self, n, grids, vertex_set, edges=None):
        """A graph (an undirected graph that is) that models 
        the update equations of agents positions on a state space 
        (defined as a grid).

        The graph has a vertex set {1,2,...,n} so defined such that 
        (i,j) is one of the graph's edges in case i and j are neighbors.
        This graph changes over time since the relationship between neighbors
        can change.

        Paramters
        =========
            .grids
            n: number of initial birds (vertices) on this graph.
            .V: vertex_set, a set of vertices {1,2,...,n} that represent the labels
            of birds in a flock. Represent this as a list (see class vertex).
            .E: edges, a set of unordered pairs E = {(i,j): i,j \in V}.
                Edges have no self-loops i.e. i≠j or repeated edges (i.e. elements are distinct).
        """
        self.N = n
        if vertex_set is None:            
            self.vertex_set = {f"{i+1}":BirdSingle(grids[i], 1, 1,\
                    None, random.random(), label=f"{i}") for i in range(n)}
        else:
            self.vertex_set = {f"{i+1}":vertex_set[i] for i in range(n)}
        
        # edges are updated dynamically during game
        self.edges_set = edges 

        # obtain the graph params
        self.reset(self.vertex_set[list(self.vertex_set.keys())[0]].w)

    def reset(self, w):
        # graph entities: this from Jadbabaie's paper
        self.Ap = np.zeros((self.N, self.N)) #adjacency matrix
        self.Dp = np.zeros((self.N, self.N)) #diagonal matrix of valencies
        self.θs = np.ones((self.N, 1))*w # agent headings
        self.I  = np.ones((self.N, self.N))
        self.Fp = np.zeros_like(self.Ap) # transition matrix for all the headings in this flock

    def insert_vertex(self, vertex):
        if isinstance(vertex, list):
            assert isinstance(vertex, BirdSingle), "vertex to be inserted must be instance of class Vertex."
            for vertex_single in vertex:
                self.vertex_set[vertex_single.label] = vertex_single.neighbors
        else:
            self.vertex_set[vertex.label] = vertex

    def insert_edge(self, from_vertices, to_vertices):
        if isinstance(from_vertices, list) and isinstance(to_vertices, list):
            for from_vertex, to_vertex in zip(from_vertices, to_vertices):
                self.insert_edge(from_vertex, to_vertex)
            return
        else:
            assert isinstance(from_vertices, BirdSingle), "from_vertex to be inserted must be instance of class Vertex."
            assert isinstance(to_vertices, BirdSingle), "to_vertex to be inserted must be instance of class Vertex."
            from_vertices.update_neighbor(to_vertices)
            self.vertex_set[from_vertices.label] = from_vertices.neighbors
            self.vertex_set[to_vertices.label] = to_vertices.neighbors

    def adjacency_matrix(self, t):
        for i in range(self.Ap.shape[0]):
            for j in range(self.Ap.shape[1]):
                for verts in sorted(self.vertex_set.keys()):
                    if str(j) in self.vertex_set[verts].neighbors:
                        self.Ap[i,j] = 1 
        return self.Ap

    def diag_matrix(self):
        "build Dp matrix"
        i=0
        for vertex, egdes in self.vertex_set.items():
            self.Dp[i,i] = self.vertex_set[vertex].valence
        return self.Dp

    def update_headings(self, t):
        return self.adjacency_matrix(t)@self.θs



In [32]:
'we will first create the grids coverage for the respective birds'

num_agents=10

# for every agent, create the grid bounds
grid_mins = [[-1, -1, -np.pi]]
grid_maxs = [[1, 1, np.pi]]   
grids = flockGrid(grid_mins, grid_maxs, dx=.1, N=101)

# sanity check
grids[0].N.T

array([[101, 101, 101]])

In [34]:
flock1 = BirdFlock(grids, num_agents=10)

Bird 0.

In [ ]:
# it feels like we also need create the case for when birds in a flock have a sparse interaction.

In [ ]:
# compute nearest neigbors on a grid

In [ ]:
# Now create classes for flocks splitting, flocks contraction, and flocks expansion.

In [ ]:
# create a flock of 10 birds for example
dubins_flock = DubinsFlock(grids, u_bound=1, w_bound=1)

In [ ]:
dubins_flock

In [ ]:

brt = np.load("../data/rcbrt.npz")['brt']
brt_time = np.load("../data/rcbrt.npz")['brt_time']
brt_time += [brt_time[-1]]
                                  
f = plt.figure(figsize=(16,9))
gs  = gridspec.GridSpec(2,2, f)


def plot_brt(ax, mesh, time_step):
	ax.grid('on')
	ax.add_collection3d(mesh)  
	xlim = (0, 1.75)
	ylim = (0, 1.75)
	zlim = (0, 2*np.pi)

	ax.set_xlim3d(*xlim)
	ax.set_ylim3d(*ylim)
	ax.set_zlim3d(*zlim)
	ax.set_title(rf'BRT at {time_step}/2.5 secs.', \
			fontdict={'fontsize':18, 'fontweight':'bold'})


# slices to plot:
slc = [0, 3, 7, 10]
brt_time = np.linspace(0, 2.0, 11)
ax = [plt.subplot(gs[0, 0], projection='3d'), 
	  plt.subplot(gs[0, 1], projection='3d'),
	  plt.subplot(gs[1, 0], projection='3d'),
	  plt.subplot(gs[1, 1], projection='3d')]
i=0
for slc_num in slc:		
	mesh = implicit_mesh(brt[slc_num], level=0, spacing=tuple(g.dx.flatten().tolist()),
									edge_color=None,  face_color='orchid')
	plot_brt(ax[i], mesh, brt_time[slc_num])
	i+=1	
plt.show()


In [ ]:
def controllability(g, gr, attr, value_func_init):
	fontdict = {'fontsize':28, 'fontweight':'bold'}
	f, (ax1, ax2) = plt.subplots(1,2,figsize=(16, 6))

	ax1.contour(g.xs[0], g.xs[1], attr, colors='red')
	ax1.set_title('Analytical TTR', fontdict =fontdict)
	ax1.set_xlabel(r"$x_1 (m)$", fontdict =fontdict)
	ax1.set_ylabel(r"$x_2 (ms^{-1})$", fontdict =fontdict)
	ax1.tick_params(axis='both', which='major', labelsize=28)
	ax1.tick_params(axis='both', which='minor', labelsize=18)
	ax1.set_xlim([-1.02, 1.02])
	ax1.set_ylim([-1.01, 1.01])
	ax1.grid()

	ax2.contour(gr.xs[0], gr.xs[1], value_func_init, colors='blue')
	ax2.set_title('Numerical TTR', fontdict =fontdict)
	ax2.set_xlabel(r"$x_1 (m)$", fontdict =fontdict)
	ax2.set_xlim([-1.02, 1.02])
	ax2.set_ylim([-1.01, 1.01])
	ax2.grid('on')
	ax2.tick_params(axis='both', which='major', labelsize=28)
	ax2.tick_params(axis='both', which='minor', labelsize=18)
	ax2.legend(loc="center left", fontsize=8) 

	f.suptitle(f"Levelsets")

	f.canvas.draw()
	f.canvas.flush_events()
	time.sleep(args.pause_time)

In [ ]:
g.xs = [g.xs[i].get() for i in range(g.dim)]
gr.xs = [gr.xs[i].get() for i in range(gr.dim)]
view_noncontrollability(g, gr, attr, value_rob)

In [ ]:
gs

In [ ]:
p1, p2, mode= obj.p1.pursuer, obj.p1.evader, 'capture'

assert isfield(p1, 'center'), 'player I must have a center '\
                                'defined for its capture equation.'
assert isfield(p2, 'center'), 'player II must have a center '\
                                'defined for its capture equation.'
assert isfield(p1, 'grid'), 'player I must have its grid info'
assert isfield(p2, 'grid'), 'player II must have its grid info'

x1 = p1.grid.xs
x2 = p2.grid.xs

p1_dyn  = cell(3)
p2_dyn = cell(3)

p1_dyn[0] = obj.vp*np.cos(x1[2])
p1_dyn[1] = obj.vp*np.sin(x1[2])
p1_dyn[2] = obj.we

p2_dyn[0] = obj.ve*np.cos(x2[2])
p2_dyn[1] = obj.ve*np.sin(x2[2])
p2_dyn[2] = obj.wp

data_agent1 = np.zeros(p1.grid.shape)
data_agent2 = np.zeros(p2.grid.shape)

if isfield(p1, 'center') and numel(p1.center==1):
    p1.center = p1.center*np.ones((p1.grid.dim,1), dtype=np.float64)
if isfield(p2, 'center') and numel(p2.center==1):
    p2.center = p2.center*np.ones((p2.grid.dim,1), dtype=np.float64)
        

### This for the Double Integrator

In [ ]:
import cupy as cp
import numpy as np

import sys
from os.path import abspath, join
sys.path.append(abspath(join('../..')))
from LevelSetPy.Utilities import *
from LevelSetPy.Grids import createGrid
from LevelSetPy.Helper import postTimeStepTTR
from LevelSetPy.Visualization import implicit_mesh
from LevelSetPy.DynamicalSystems import DoubleIntegrator
from LevelSetPy.SpatialDerivative import upwindFirstWENO5
from LevelSetPy.ExplicitIntegration import artificialDissipationGLF
from LevelSetPy.ExplicitIntegration.Integration import odeCFL3, odeCFLset
from LevelSetPy.ExplicitIntegration.Term import termRestrictUpdate, termLaxFriedrichs


In [ ]:
gmin = np.array(([[-1, -1]]),dtype=np.float64).T
gmax = np.array(([[1, 1]]),dtype=np.float64).T
g = createGrid(gmin, gmax, 101, None)

eps_targ = 1.0
u_bound = 1
target_rad = .2 #eps_targ * np.max(g.dx)
dint = DoubleIntegrator(g, u_bound)
value_time2reach = dint.min_time2reach() - target_rad
value_time2reach = np.maximum(0, value_time2reach)

In [ ]:
above_curve=dint.grid.xs[0]>dint.Gamma
below_curve=dint.grid.xs[0]<dint.Gamma
on_curve=dint.grid.xs[0]==dint.Gamma

reach_term1  = (dint.grid.xs[1] + np.emath.sqrt(4*dint.grid.xs[0] + \
                         2 * dint.grid.xs[1]**2))*above_curve
reach_term2 =  (-dint.grid.xs[1]+np.emath.sqrt(-4*dint.grid.xs[0] + \
                2 * dint.grid.xs[1]**2) )*below_curve
reach_term3 = np.abs(dint.grid.xs[1]) * on_curve
reach_time = reach_term1.real + reach_term2.real + reach_term3

In [ ]:
dataIn = cp.arange(1, 13).reshape(3, 4)
print('dataIn\n', dataIn)
indices =[ [3], [5]]
print()
print(dataIn[cp.ix_(*indices)])

### Multiple vehicles

In [ ]:

obj = Bundle({})


pdDims = 2; N = 100
v, w = +1, +1
obj.ve, obj.vp = v, v
obj.we, obj.wp = -w, w


# get player (pursuer) 1's state space
gmin = np.array(([[-5, -5, -pi]])).T
gmax = np.array(([[0, 0, pi]])).T
obj.p1 = Bundle({'pursuer':Bundle({}), 'evader':Bundle({})})
obj.p1.pursuer.grid = createGrid(gmin, gmax, N, pdDims)
obj.p1.pursuer.center = np.array(([[-2.5, -2.5, 0]]),dtype=np.float64).T
obj.p1.pursuer.radius = 0.5
obj.p1.pursuer.basis = np.array(([[1,0,0]]))

# get player (evader) 2's state space
gmin = np.array(([[0, 0, pi]])).T
gmax = np.array(([[5, 5, 3*pi]])).T
obj.p1.evader.grid = createGrid(gmin, gmax, N, pdDims)
obj.p1.evader.center = np.array(([[2.5, 2.5, 2*pi]]),dtype=np.float64).T
obj.p1.evader.radius = .5
obj.p1.evader.basis = np.array(([[0,1,0]]))

# get player (pursuer) 3's state space
gmin = np.array(([[5, 5, 3*pi]])).T
gmax = np.array(([[10, 10, 5*pi]])).T
obj.p2 = Bundle({'pursuer':Bundle({}), 'evader':Bundle({})})
obj.p2.pursuer.grid = createGrid(gmin, gmax, N, pdDims)
obj.p2.pursuer.center = np.array(([[7.5, 7.5, 4*pi]]),dtype=np.float64).T
obj.p2.pursuer.radius = .5
obj.p2.pursuer.basis = np.array(([[0,0,1]]))

# get player (evader) 4's state space
gmin = np.array(([[10, 10, 5*pi]])).T
gmax = np.array(([[15, 15, 7*pi]])).T
obj.p2.evader.grid = createGrid(gmin, gmax, N, pdDims)
obj.p2.evader.center = np.array(([[12.5, 12.5, 6*pi]]),dtype=np.float64).T
obj.p2.evader.radius = .5
obj.p2.evader.basis = np.array(([[0,0,0,1]]))

# Full grid
gmin = np.array(([[-5, -5, -pi]])).T
gmax = np.array(([[15, 15, 7*pi]])).T
obj.full_grid = createGrid(gmin, gmax, N, pdDims)


In [ ]:
obj.p1.pursuer.xdot = dubins_absolute(obj, obj.p1.pursuer)
obj.p1.evader.xdot  = dubins_absolute(obj, obj.p1.evader)
obj.p2.pursuer.xdot = dubins_absolute(obj, obj.p2.pursuer)
obj.p2.evader.xdot  = dubins_absolute(obj, obj.p2.evader)
 
value_func = shapeRectangleByCorners(obj.full_grid, lower=-3, upper=13)

In [ ]:
# we now have a large value function, decompose the value w.r.t to the
# basis of the four vehicles to get its correspondiung decomposition into diff bases

In [ ]:
# compose rhe full grid as a tensor of all players
obj.grid_compose_vs = obj.p1.pursuer.grid.vs + obj.p1.evader.grid.vs + \
                      obj.p2.pursuer.grid.vs + obj.p2.evader.grid.vs
print([x.shape for x in obj.grid_compose_vs])

# too big
#FullTensor = np.meshgrid(*obj.grid_compose_vs, indexing='ij')

In [ ]:
# direction cosine of p1.pursuer w.r.t p1.evader
obj.p1.pursuer.basis_p1evader = np.dot(obj.p1.pursuer.basis.T, obj.p1.evader.basis)

In [ ]:
p1_pursuer = np.asarray(obj.p1.pursuer.grid.xs).transpose([1,2,3,0])
p1_evader = np.asarray(obj.p1.evader.grid.xs).transpose([1,2,3,0])
p2_pursuer = np.asarray(obj.p2.pursuer.grid.xs).transpose([1,2,3,0])
p2_evader = np.asarray(obj.p2.evader.grid.xs).transpose([1,2,3,0])
FT = np.concatenate((p1_pursuer, p1_evader), )
FT.shape

# use HOSVD to compute optimal rotation vector of one agent's state space to another's state space

def kabsch(P, Q):
    """
    Using the Kabsch algorithm with two sets of paired point P and Q, centered
    around the centroid. Each vector set is represented as an NxD
    matrix, where D is the the dimension of the space.
    The algorithm works in three steps:
    - a centroid translation of P and Q (assumed done before this function
      call)
    - the computation of a covariance matrix C
    - computation of the optimal rotation matrix U
    For more info see http://en.wikipedia.org/wiki/Kabsch_algorithm
    Parameters
    ----------
    P : array
        (N,D) matrix, where N is points and D is dimension.
    Q : array
        (N,D) matrix, where N is points and D is dimension.
    Returns
    -------
    U : matrix
        Rotation matrix (D,D)
    """

    # Computation of the covariance matrix
    C = np.dot(np.transpose(P), Q)

    # Computation of the optimal rotation matrix
    # This can be done using singular value decomposition (SVD)
    # Getting the sign of the det(V)*(W) to decide
    # whether we need to correct our rotation matrix to ensure a
    # right-handed coordinate system.
    # And finally calculating the optimal rotation matrix U
    # see http://en.wikipedia.org/wiki/Kabsch_algorithm
    V, S, W = np.linalg.svd(C)
    d = (np.linalg.det(V) * np.linalg.det(W)) < 0.0

    if d:
        S[-1] = -S[-1]
        V[:, -1] = -V[:, -1]

    # Create Rotation matrix U
    U = np.dot(V, W)

    return U


In [ ]:
a = np.arange(9).reshape(3,3)
b, c = 2*a, 3*a

In [ ]:
np.minimum(a, b, c)

In [ ]:
a = a.flatten()
a[1:-1]